# Выбор локации для скважины

## Обзор данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [4]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
data_0.info()
data_0.shape
data_0.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
data_0['id'].duplicated().sum()

10

In [7]:
data_1.info()
data_1.shape
data_1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [8]:
data_1['id'].duplicated().sum()

4

In [9]:
data_2.info()
data_2.shape
data_2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [10]:
data_2['id'].duplicated().sum()

4

In [11]:
# id не несет нужной для расчетов информации, можно удалить
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

## Обучение моделей для каждого региона

Целевой признак в данном случае 'Product'

In [12]:
pd.options.mode.chained_assignment = None # добавляем код, чтобы не появлялось предупреждение
def region_prediction(data):
    features = data.drop(["product"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, train_size=0.75, random_state=12345)
    scaler = StandardScaler()
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler() # создадим объект этой структуры данных
    scaler.fit(features_train[numeric]) # настроим объект на обучающих данных 
    features_train[numeric] = scaler.transform(features_train[numeric]) # проеобразуем обучающую выборку
    features_valid[numeric] = scaler.transform(features_valid[numeric]) # преобразуем валидационную выборку 
    
    lr = LinearRegression(normalize=False)
    lr.fit(features_train, target_train)
    predictions = lr.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    stock_mean = data['product'].mean() # средний фактический запас сырья в регионе
    stock_mean_pred = predictions.mean() # средний запас предсказанного сырья в регионе
    return predictions, rmse, stock_mean, stock_mean_pred, target_valid.reset_index(drop=True)

In [13]:
pred_0, rmse_0, stock_mean_0, stock_mean_pred_0, target_valid_0 = region_prediction(data_0)
print('RMSE модели в регионе 0 = {:.3f}'.format(rmse_0))
print('Средний запас предсказанного сырья 0 = {:.3f} т. баррелей'.format(stock_mean_pred_0))

RMSE модели в регионе 0 = 37.579
Средний запас предсказанного сырья 0 = 92.593 т. баррелей


/Users/zenazujkova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [14]:
pred_1, rmse_1, stock_mean_1, stock_mean_pred_1, target_valid_1 = region_prediction(data_1)
print('RMSE модели в регионе 1 = {:.3f}'.format(rmse_1))
print('Средний запас предсказанного сырья 1 = {:.3f} т. баррелей'.format(stock_mean_pred_1))

RMSE модели в регионе 1 = 0.893
Средний запас предсказанного сырья 1 = 68.729 т. баррелей


/Users/zenazujkova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


In [15]:
pred_2, rmse_2, stock_mean_2, stock_mean_pred_2, target_valid_2 = region_prediction(data_2)
print('RMSE модели в регионе 2 = {:.3f}'.format(rmse_2))
print('Средний запас предсказанного сырья 2 = {:.3f} т. баррелей'.format(stock_mean_pred_2))

RMSE модели в регионе 2 = 40.030
Средний запас предсказанного сырья 2 = 94.965 т. баррелей


/Users/zenazujkova/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


### Вывод:
Лучший показатель RMSE модели в Регионе 1 (RMSE = 0.893), этот регион наиболее предсказуем. 

Средний запас предсказанного сырья 1 = 68.729 т. баррелей, что меньше чем в двух других регионах(92.593 и 94.965)

В регионах 0 и 2 высокий показатель RMSE, значит предсказания модели хуже. Но средний запас предсказанного сырья выше.


## Подготовка к расчету прибыли

In [17]:
BUDGET = 10_000_000_000 #выделяемый бюджет
BOREHOLE_NUMBER = 200 #кол-во скважин для разработки
INCOME_UNIT = 450_000 #доход с единицы сырья
BOREHOLE_BUDGET = BUDGET/BOREHOLE_NUMBER #бюджет на каждую скважину
QUANTITY_BARREL = BOREHOLE_BUDGET/INCOME_UNIT #кол-во тысяч баррелей с одной скважины для окупаемости
print('Выделяемый бюджет для разработки:',BUDGET)
print('Кол-во скважин для разработки:',BOREHOLE_NUMBER)
print('Бюджет на каждую скважину:',BOREHOLE_BUDGET)
print('Кол-во тысяч баррелей с одной скважины для безубыточности: {:.3f}'.format(QUANTITY_BARREL))

Выделяемый бюджет для разработки: 10000000000
Кол-во скважин для разработки: 200
Бюджет на каждую скважину: 50000000.0
Кол-во тысяч баррелей с одной скважины для безубыточности: 111.111


In [18]:
# выведем на экран средний запас сырья в регионах
print('Средний запас сырья региона 0 = {:.3f} т. баррелей'.format(stock_mean_0))
print('Средний запас сырья региона 1 = {:.3f} т. баррелей'.format(stock_mean_1))
print('Средний запас сырья региона 2 = {:.3f} т. баррелей'.format(stock_mean_2))

Средний запас сырья региона 0 = 92.500 т. баррелей
Средний запас сырья региона 1 = 68.825 т. баррелей
Средний запас сырья региона 2 = 95.000 т. баррелей


### Вывод: 
Среднее кол-во тысяч барреллей для безубыточности составляет около 111, что превышает средний фактический запас в каждом регионе. Самый маленький запас в Месторождении 1.  

In [23]:
# функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(prediction, target):
    data = pd.concat([prediction, target],axis=1)
    data.columns = ['prediction','target']
    data = data.sort_values(by = 'prediction', ascending = False)[:BOREHOLE_NUMBER]
    return (data['target'].sum() * INCOME_UNIT - BUDGET)

In [24]:
# расчет прибыли для региона 0
revenue_0 = profit(pred_0, target_valid_0)

print('Прибыль для полученного объёма сырья региона 0 = {} млн. руб'.format(revenue_0/ 10e6))

Прибыль для полученного объёма сырья региона 0 = 332.0826043139851 млн. руб


In [25]:
# расчет прибыли для региона 1
revenue_1 = profit(pred_1, target_valid_1)

print('Прибыль для полученного объёма сырья региона 1 = {} млн. руб'.format(revenue_1 / 10e6))

Прибыль для полученного объёма сырья региона 1 = 241.5086696681511 млн. руб


In [26]:
# расчет прибыли для региона 2
revenue_2 = profit(pred_2, target_valid_2)

print('Прибыль для полученного объёма сырья региона 2 = {} млн. руб'.format(revenue_2 / 10e6))

Прибыль для полученного объёма сырья региона 2 = 271.03499635998327 млн. руб


## Функция для расчетов прибыли и рисков для каждого региона с применением техники Bootstrap

In [27]:
def estimate(prediction, target):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = prediction[target_subsample.index]
        values.append(profit(pred_subsample, target_subsample))
    values = pd.Series(values)
    mean = np.mean(values) / 10e6
    lower = values.quantile(0.025) / 10e6
    upper = values.quantile(0.975) / 10e6
    confidence_interval = (lower, upper)
    risk_of_loss = (values < 0).mean()
    
    print('Средняя прибыль = {:.2f} млн.руб.'.format(mean))
    print('95% доверительный интервал от {:.2f} до {:.2f} млн.руб.'.format(lower, upper))
    print('Процент риска {:.1%}'.format(risk_of_loss))

In [28]:
region_0 = estimate(pred_0, target_valid_0)

Средняя прибыль = 39.62 млн.руб.
95% доверительный интервал от -11.12 до 90.98 млн.руб.
Процент риска 6.9%


In [29]:
region_1 = estimate(pred_1, target_valid_1)

Средняя прибыль = 45.60 млн.руб.
95% доверительный интервал от 3.38 до 85.23 млн.руб.
Процент риска 1.5%


In [30]:
region_2 = estimate(pred_2, target_valid_2)

Средняя прибыль = 40.44 млн.руб.
95% доверительный интервал от -16.34 до 95.04 млн.руб.
Процент риска 7.6%


## Общий вывод
Изучили данные трех регионов. На основании предоставленных данных обучили модель линейной регрессии. 
Рассчитали прибыль и риски для каждого региона. 
Не смотря на то, что средний запас предсказанного сырья в регионе 1 = 68.729 т. баррелей, данный регион является наиболее перспективным. Добыча нефти в этом регионе связана с наименьшим риском и предполагается получить наибольшую среднюю прибыль в размере 45.60 млн.руб.

Регион 0 и регион 2 имеют вероятность риска в 6.9% и 7.6% соответственно. В регионе 1 данный процент составляет лишь 1.5. 